# Comparing different ways of referencing init functions

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [11]:
#make sample dataset
import numpy as np
from sklearn.datasets import make_classification
n_classes = 2

# (X,y): Original distribution
X, y = make_classification(n_samples=1000,
                           n_classes=n_classes, 
                           n_features=300,
                           n_informative=300,
                           n_redundant=0,
                           n_repeated=0,
                           random_state=15
                          )
# One-hot encoding of the label
y = np.eye(n_classes)[y]


#define subclass
from typing import Any
import sys

from tensorflow.keras import Model as Keras_Model
from safemodel import SafeModel

class KModel_Subclass_both_underscore_init(Keras_Model ):
    def __init__(self,inputs,outputs) -> None:
        print('in init function of class KModel_Subclass_outer_underscore_init')
        def call(self, inputs,outputs):
            pass
        """Creates model and applies constraints to params"""
        Keras_Model.__init__(self,inputs,outputs)
        
        
class KModel_Subclass_neither_underscore_init(Keras_Model ):
    def init(self,inputs,outputs) -> None:
        print('in init function of class KModel_Subclass_neither_underscore_init')
        def call(self, inputs,outputs):
            pass
        """Creates model and applies constraints to params"""
        Keras_Model.init(self,inputs,outputs)        
      
class KModel_Subclass_inner_underscore_init(Keras_Model ):

    def init(self,inputs,outputs) -> None:
        print('in init function of class KModel_Subclass_inner_underscore_init')
        def call(self, inputs,outputs):
            pass
        """Creates model and applies constraints to params"""
        Keras_Model.__init__(self,inputs,outputs) 
        
class KModel_Subclass_outer_underscore_init(Keras_Model ):
    def __init__(self,inputs,outputs) -> None:
        print('in init function of class KModel_Subclass_outer_underscore_init')
        def call(self, inputs,outputs):
            pass
        """Creates model and applies constraints to params"""
        Keras_Model.init(self,inputs,outputs)      
    
     

#define simple model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import CategoricalCrossentropy


the_input = Input(shape = X[0].shape)
x = Dense(128, activation='relu')(the_input)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
the_output = Dense(2, activation='softmax')(x)


variant_names= ('both', 'neither', 'only inner', 'only outer')
variants = (KModel_Subclass_both_underscore_init,
            KModel_Subclass_neither_underscore_init,
            KModel_Subclass_inner_underscore_init,
            KModel_Subclass_outer_underscore_init
           )

for variant in (0, 1, 2, 3):
    print(f'\n====== underscores in {variant_names[variant]} init function calls')
    #create two instances
    try:
        first_instance = variants[variant](inputs=the_input, outputs= the_output)
        print('Made first instance',first_instance)
        print(first_instance.summary())
        second_instance = variants[variant](inputs=the_input, outputs= the_output)
        print('Made second instance',second_instance)
        print(second_instance.summary())

        keys1 = list(first_instance.__dict__.keys())
        keys2 = list(second_instance.__dict__.keys())
        dicts_are_same = True
        for key in keys1:
            if key not in keys2:
                #print(f'key {key} present in first instance but not second')
                dicts_are_same = False
        for key2 in keys2:
            if key2 not in keys1:
                #print(f'key {key2} present in second instance but not first')
                dicts_are_same = False
        print(f' It is {dicts_are_same} that the two objects have the same set of attribute keys in their dicts.')
        if not dicts_are_same:
            print('Therefore not attempting to fit instance as it will fail')
        else:
            print('Compiling and fitting object one')
            try:
                first_instance.compile (loss='categorical_crossentropy')
                first_instance.fit(X,y)
            except AttributeError as e:
                print(f'Attempt failed with AttributeError {e}')
            print('Compiling and fitting object two')
            try:
                second_instance.compile (loss='categorical_crossentropy')
                second_instance.fit(X,y)
            except AttributeError as e:
                print(f'Attempt failed with AttributeError {e}') 
    except Exception as e:
        print(f'Attempt to create instances using underscores in {variant_names[variant]} init names '
               f'failed with exception:\n {e}')          



====== underscores in both init function calls
in init function of class KModel_Subclass_outer_underscore_init
Made first instance <__main__.KModel_Subclass_both_underscore_init object at 0x297a53460>
Model: "k_model__subclass_both_underscore_init_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300)]             0         
                                                                 
 dense_12 (Dense)            (None, 128)               38528     
                                                                 
 dense_13 (Dense)            (None, 128)               16512     
                                                                 
 dense_14 (Dense)            (None, 64)                8256      
                                                                 
 dense_15 (Dense)            (None, 2)                 130       
                      

# Version for gist

In [12]:
#make sample dataset
import numpy as np
from sklearn.datasets import make_classification
n_classes = 2

# (X,y): Original distribution
X, y = make_classification(n_samples=1000,
                           n_classes=n_classes, 
                           n_features=300,
                           n_informative=300,
                           n_redundant=0,
                           n_repeated=0,
                           random_state=15
                          )
# One-hot encoding of the label
y = np.eye(n_classes)[y]


#define subclass
from typing import Any
import sys

from tensorflow.keras import Model as Keras_Model


class KModel_Subclass(Keras_Model ):
    def __init__(self,inputs,outputs) -> None:
        print('in init function of class KModel_Subclass')
        def call(self, inputs,outputs):
            super().call()
            pass
        
        Keras_Model.__init__(self,inputs,outputs)
        

#define simple model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import CategoricalCrossentropy

# define layers for two model instances 
# using different instances of layers objects
# to avoid any unintentional interaction effects
the_input = Input(shape = X[0].shape)
x = Dense(128, activation='relu')(the_input)
x = Dense(64, activation='relu')(x)
the_output = Dense(2, activation='softmax')(x)

the_input2 = Input(shape = X[0].shape)
x2 = Dense(128, activation='relu')(the_input2)
x2 = Dense(64, activation='relu')(x2)
the_output2 = Dense(2, activation='softmax')(x2)


first_instance = KModel_Subclass(inputs=the_input, outputs= the_output)
print('Made first instance',first_instance)

second_instance = KModel_Subclass(inputs=the_input2, outputs= the_output2)
print('Made second instance',second_instance)


print('--------------------------------------------\n')
print('Testing whether the two instances has the same attributes, i.e. set of keys in their __dict__')
keys1 = list(first_instance.__dict__.keys())
keys2 = list(second_instance.__dict__.keys())
in_first_only = []

dicts_are_same = True
for key in keys1:
    if key not in keys2:
        #print(f'key {key} present in first instance but not second')
        in_first_only.append(key)
        dicts_are_same = False
if len(in_first_only)>0:
    print('These attributes are present in the first instance but missing from the second:')
    for val in in_first_only:
        print(f'\t{val}')

print(f'\n==> After initialisation it is {dicts_are_same} that the two objects have the same set of attribute keys in their dicts.')

print('--------------------------------------------\n')
print('Trying to call each instance summary() method  ')
try:
    print('instance 1:')
    first_instance.summary()
    print('instance2')
    second_instance.summary()
except Exception as e:
    print(f'failed with reason: {e}')

print('--------------------------------------------\n')
print('Compiling and fitting object one')
try:
    first_instance.compile (loss='categorical_crossentropy')
    first_instance.fit(X,y)
    print('first object successfully fitted')
except AttributeError as e:
    print(f'Attempt failed with AttributeError {e}')

print('\nCompiling and fitting object two')
try:
    second_instance.compile (loss='categorical_crossentropy')
    second_instance.fit(X,y)
    print('second object successfully fitted')
except AttributeError as e:
    print(f'Attempt failed with AttributeError {e}') 


in init function of class KModel_Subclass
Made first instance <__main__.KModel_Subclass object at 0x2c85c0f10>
in init function of class KModel_Subclass
Made second instance <__main__.KModel_Subclass object at 0x297c3e1f0>
--------------------------------------------

Testing whether the two instances has the same attributes, i.e. set of keys in their __dict__
These attributes are present in the first instance but missing from the second:
	_nested_inputs
	_nested_outputs
	_enable_dict_to_input_mapping
	_input_layers
	_output_layers
	_input_coordinates
	_output_coordinates
	_output_mask_cache
	_output_tensor_cache
	_output_shape_cache
	_network_nodes
	_nodes_by_depth
	_layer_call_argspecs
	_feed_input_names
	_feed_inputs
	_feed_input_shapes
	_tensor_usage_count

==> After initialisation it is False that the two objects have the same set of attribute keys in their dicts.
--------------------------------------------

Trying to call each instance summary() method  
instance 1:
Model: "k_mo

# specify layers in init as suggested here: 
https://www.tensorflow.org/api_docs/python/tf/keras/Model#args_1
where they say 
> There are two ways to instantiate a Model:
...
2 - By subclassing the Model class: in that case, you should define your layers in __init__() and you should implement the model's forward pass in call().


````import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)

model = MyModel()
````

Obnvously this is a route we would prefer not to go down as it is miuch less general and harder to work with

But checking it anyway ...

In [16]:
import tensorflow as tf
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(2, activation=tf.nn.softmax)
    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)
model1 = MyModel()
model2= MyModel()

In [19]:
keys1 = list(model1.__dict__.keys())
keys2 = list(model2.__dict__.keys())
in_first_only = []

dicts_are_same = True
for key in keys1:
    if key not in keys2:
        #print(f'key {key} present in first instance but not second')
        in_first_only.append(key)
        dicts_are_same = False
if len(in_first_only)>0:
    print('These attributes are present in the first instance but missing from the second:')
    for val in in_first_only:
        print(f'\t{val}')

print(f'\n==> After initialisation it is {dicts_are_same} that the two objects have the same set of attribute keys in their dicts.')
#print(keys1)



==> After initialisation it is True that the two objects have the same set of attribute keys in their dicts.


In [20]:
model1.compile (loss='categorical_crossentropy')
print('model1 compiled')
model1.fit(X,y)
print('model1 fitted')

model2.compile (loss='categorical_crossentropy')
model2.fit(X,y)
print('model2 fitted')

model1 compiled
32/32 [==============================] - 0s 7ms/step - loss: 1.8015
model1 fitted
32/32 [==============================] - 0s 7ms/step - loss: 1.8151
model2 fitted
